In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.cluster import DBSCAN
from sklearn.neighbors import NearestNeighbors

In [2]:
df = pd.read_csv('../../data/telecom_pca.csv')
x = np.array(df.drop(columns=['TARGET']))

In [3]:
db = DBSCAN(eps=10, min_samples=5).fit(x)

In [4]:
db.labels_

array([-1, -1, -1, ..., -1, -1, -1], dtype=int64)

In [6]:
# Plot the clusters
#plt.scatter(x[:, 0], x[:,1], c = db.labels_, cmap= "plasma") # plotting the clusters
#plt.xlabel("Income") # X-axis label
#plt.ylabel("Spending Score") # Y-axis label
#plt.show() # showing the plot